In [26]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import bs4 as bs4
from urllib.parse import urlparse
import requests
from collections import Counter
import pandas as pd
import re

In [27]:
df = pd.read_csv('../website_classification.csv')
df

,website_url,cleaned_website_text,Category
0,https://www.fubo.tv/lp/StreamTV/?irad=343747&i...,stream live tv sports fubotv free trial sport ...,Streaming Services
1,https://www.hulu.com/,stream tv movie live online hulu watch tv show...,Streaming Services
2,https://tv.youtube.com/,youtube tv watch dvr live sports shows news st...,Streaming Services
3,https://www.attwatchtv.com/,watchtv stream channel live tv want watchtv ex...,Streaming Services
4,https://www.klowdtv.com/,live tv streaming klowdtv klowdtv provide live...,Streaming Services
...,...,...,...
1089,https://www.slack.com,slack is your productivity platform slackslack...,Social Networking and Messaging
1090,https://www.discord.com,discord your place to talk and hang outdiscord...,Social Networking and Messaging
1091,https://www.dribbble.com,dribbble discover the worlds top designers cre...,Social Networking and Messaging
1092,https://www.producthunt.com,product hunt the best new products in tech pr...,Social Networking and Messaging


In [71]:
from bs4 import BeautifulSoup
import bs4 as bs4
from urllib.parse import urlparse
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os


class ScrapTool:   
    def visit_url(self,website_url):
        '''
        Visit URL. Download the Content. Initialize the beautifulsoup object. Call parsing methods. Return Series object.
        '''
        #Set up Selenium webdriver
        PATH = 'C:/Users/User01/Downloads/chromedriver_win32/chromedriver.exe'

        # Initialize the Chrome webdriver and pass the Chrome options
        service = Service(PATH)
        driver = webdriver.Chrome(service = service)

        #Load website
        driver.get(website_url)

        #Handle captcha
        captcha = None
        try:
            captcha = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.recaptcha-checkbox-border")))
        except:
            pass
        
        # Check if the cookie consent button is present
        cookie_button = None
        try:
            cookie_button = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "cookie-consent-button")))
        except:
            pass

        # Handle cookies if the button is present
        if cookie_button:
            cookie_button.click()

        # Extract HTML content
        html_content = driver.page_source
        driver.quit()

        soup = BeautifulSoup(html_content,'lxml')
        result = {
            "website_url": website_url,
            "website_name": self.get_website_name(website_url),
            "website_text": self.get_html_title_tag(soup)+self.get_html_meta_tags(soup)+self.get_html_heading_tags(soup)+
                                    self.get_text_content(soup)
        }
        
        #Convert to Series object and return
        return pd.Series(result)
    
    def get_website_name(self,website_url):
        '''
        Example: returns "google" from "www.google.com"
        '''
        return "".join(urlparse(website_url).netloc.split(".")[-2])
    
    def get_html_title_tag(self,soup):
        '''Return the text content of <title> tag from a webpage'''
        return '. '.join(soup.title.contents)
    
    def get_html_meta_tags(self,soup):
        '''Returns the text content of <meta> tags related to keywords and description from a webpage'''
        tags = soup.find_all(lambda tag: (tag.name=="meta") & (tag.has_attr('name') & (tag.has_attr('content'))))
        content = [str(tag["content"]) for tag in tags if tag["name"] in ['keywords','description']]
        return ' '.join(content)
    
    def get_html_heading_tags(self,soup):
        '''returns the text content of heading tags. The assumption is that headings might contain relatively important text.'''
        tags = soup.find_all(["h1","h2","h3","h4","h5","h6"])
        content = [" ".join(tag.stripped_strings) for tag in tags]
        return ' '.join(content)
    
    def get_text_content(self,soup):
        '''returns the text content of the whole page with some exception to tags. See tags_to_ignore.'''
        tags_to_ignore = ['style', 'script', 'head', 'title', 'meta', '[document]',"h1","h2","h3","h4","h5","h6","noscript"]
        tags = soup.find_all(text=True)
        result = []
        for tag in tags:
            stripped_tag = tag.strip()
            if tag.parent.name not in tags_to_ignore\
                and isinstance(tag, bs4.element.Comment)==False\
                and not stripped_tag.isnumeric()\
                and len(stripped_tag)>0:
                result.append(stripped_tag)
        return ' '.join(result)

In [72]:
def cleaning_text(text):
    text = text.lower()
    text = text.replace('\n',' ')
    text = text.replace('\t',' ')
    text = text.replace('\r',' ')
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r' +',' ',text)
    return text

In [73]:
def content_generation(website):
    try:
        scrapTool = ScrapTool()
        web = scrapTool.visit_url(website)
        text = cleaning_text(web['website_text'])
        return text
    except:
        print('Error: ',website)
        return None

In [79]:
websites = [
    "https://www.coinbase.com/?clickId=2KURQM0mdxyNR%3ANV62XVyyixUkF0XH05%3A0PexM0&utm_source=impact&utm_medium=growthp&utm_campaign=rt_p_m_w_d_acq_imp_gro_aff_GACMEDIA&utm_content=552039&utm_creative=Online%20Tracking%20Link&irgwc=1"
]


for website in websites:
    text = content_generation(website)
    if text is not None:
        print(text)
        web = [website,text,'Computers and Technology']
        df.loc[len(df)] = web

coinbase buy sell bitcoin ethereum and more with trustcoinbase is a secure platform that makes it easy to buy sell and store cryptocurrency like bitcoin ethereum and more based in the usa coinbase is available in over countries worldwidejump start your crypto portfolio coinbase is the easiest place to buy and sell cryptocurrency sign up and get started today explore web profiles and claim one for free earn up to worth of crypto create your cryptocurrency portfolio today manage your portfolio recurring buys vault protection mobile apps the most trusted cryptocurrency platform secure storage protected assets industry best practices get started in a few minutes coinbase supports a variety of the most popular digital currencies earn up to worth of crypto company learn individuals businesses developers support hello singaporeget started jump start your portfolio jump start your crypto portfolio get started create and customize your web profile check out other profiles and explore popular nf

In [80]:
df[df["website_url"]=="https://www.coinbase.com/?clickId=2KURQM0mdxyNR%3ANV62XVyyixUkF0XH05%3A0PexM0&utm_source=impact&utm_medium=growthp&utm_campaign=rt_p_m_w_d_acq_imp_gro_aff_GACMEDIA&utm_content=552039&utm_creative=Online%20Tracking%20Link&irgwc=1"]

,website_url,cleaned_website_text,Category
1093,https://www.coinbase.com/?clickId=2KURQM0mdxyN...,coinbase buy sell bitcoin ethereum and more wi...,Computers and Technology


In [81]:
df['Category'].value_counts()

Computers and Technology           258
Social Networking and Messaging    114
Business/Corporate                 107
Streaming Services                 103
Sports                             100
E-Commerce                         100
Games                               99
News                                93
Law and Government                  83
Adult                               21
Forums                              16
Name: Category, dtype: int64

In [78]:
df.reset_index(drop=True,inplace=True)

In [82]:
df.to_csv("website_classification.csv",index=False)